# Financial Transactions Data Cleaning

In [1]:
import pandas as pd
import numpy as np

## Load Data

In [ ]:
data = pd.read_csv('../data/raw/dirty_financial_transactions.csv')
df = data.copy()

<bound method DataFrame.isnull of       Transaction_ID Transaction_Date Customer_ID    Product_Name  Quantity  \
0              T0001       2024-08-02       C2205      Headphones      -5.0   
1              T0002       2020-02-10       C3156         Coffee      469.0   
2              T0003       2025-02-30       C2919          Tablet      -4.0   
3              T0004       2020-08-17       C3009             Tab      -7.0   
4              T0005       2025-02-30       C3488  Coffee Machine     -10.0   
...              ...              ...         ...             ...       ...   
99995            NaN       2021-10-06       C1743      Headphones      -8.0   
99996         T99997       2024-08-25       C4830      Smartphone       NaN   
99997         T99998       2023-13-01        C280          Laptop     -10.0   
99998         T99999       2020-07-12       C4059      Headphones      10.0   
99999        T100000       2023-10-04       C1805          Tablet       2.0   

                 

## Remove Duplicates and Missing Values

In [10]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.replace(['', 'NA', 'N/A', 'null', None], np.nan, inplace=True)
df.dropna(inplace=True)

## Standardize Payment Method

In [11]:
df['Payment_Method'] = df['Payment_Method'].str.strip().str.lower().str.replace(' ', '', regex=False)
final_standard = {
    'creditcard': 'credit_card',
    'paypal': 'paypal',
    'cash': 'cash',
}
df['Payment_Method'] = df['Payment_Method'].replace(final_standard)

## Clean Price and Quantity

In [12]:
df['Price'] = df['Price'].astype(str).str.replace('$', '', regex=False).astype(float)
df['Price'] = np.abs(df['Price'])
df['Quantity'] = np.abs(df['Quantity'])

## Fix Product Names

In [13]:
from rapidfuzz import process
correct_names = ['Tablet', 'Laptop', 'Coffee Machine', 'Smartphone', 'Headphones']
def fix_name(name):
    match, score, _ = process.extractOne(name, correct_names)
    return match if score > 50 else name
df['Product_Name'] = df['Product_Name'].apply(fix_name)
df['Product_Name'] = df['Product_Name'].str.title()

## Standardize Transaction Status

In [14]:
df['Transaction_Status'] = df['Transaction_Status'].str.strip().str.title()

## Drop Duplicate Transaction IDs

In [15]:
df = df.drop_duplicates(subset=['Transaction_ID'], keep='first')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44905 entries, 1 to 99993
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction_ID      44905 non-null  object 
 1   Transaction_Date    44905 non-null  object 
 2   Customer_ID         44905 non-null  object 
 3   Product_Name        44905 non-null  object 
 4   Quantity            44905 non-null  float64
 5   Price               44905 non-null  float64
 6   Payment_Method      44905 non-null  object 
 7   Transaction_Status  44905 non-null  object 
dtypes: float64(2), object(6)
memory usage: 3.1+ MB


## Product Counts

In [16]:
product_counts = df['Product_Name'].value_counts().reset_index()
product_counts.columns = ['Product_Name', 'Count']
product_counts

,Product_Name,Count
0,Tablet,9151
1,Smartphone,8969
2,Coffee Machine,8949
3,Laptop,8938
4,Headphones,8898


## Save Cleaned Data

In [17]:
import os
output_path = '../data/processed/cleaned_financial_transactions.csv'
if os.path.exists(output_path):
    os.remove(output_path)
df.to_csv(output_path, index=False)

## Data Shapes

In [18]:
data.shape

(100000, 8)

In [19]:
df.shape

(44905, 8)